In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [4]:
digits = pd.read_csv ("../downloads/datasets/train.csv")
test = pd.read_csv ("../downloads/datasets/test.csv")

y=digits.label


X=digits.drop(['label'], axis=1)

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.20)

train_data = np.array(X_train, dtype=np.float32).reshape(-1,28,28,1)
train_labels = np.asarray(y_train, dtype=np.int32)
eval_data = np.array(X_test, dtype=np.float32)
eval_labels = np.asarray(y_test, dtype=np.int32)

test = np.array(test, dtype=np.float32)

In [5]:
datagen= ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  


datagen.fit(train_data)

In [6]:
i=0
for x_train_augmented, y_train_augmented in datagen.flow(train_data, train_labels, batch_size=33600):
        x_train_augmented = x_train_augmented.reshape(-1, 28, 28, 1)
        i += 1
        if i > 2:
             break

In [7]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
  loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.000000001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [8]:
def main(unused_argv):

    mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_AO")
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x":x_train_augmented},
      y=y_train_augmented,
      batch_size=100,
      num_epochs=30,
      shuffle=True)
    mnist_classifier.train(
      input_fn=train_input_fn,
      steps=1000)
   
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    mnist_classifier.train(
      input_fn=train_input_fn,
      steps=1000)

    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

    test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test},
      num_epochs=1,
      shuffle=False)
    test_predictions = list(mnist_classifier.predict(input_fn=test_input_fn))
    predicted_labels = [p["classes"] for p in test_predictions]
    
    out_file = open("cnn_tensorflow_predictions.csv", "w")
    out_file.write("ImageId,Label\n")
    for i in range(len(predicted_labels)):
        out_file.write(str(i+1) + "," + str(int(predicted_labels[i])) + "\n")
    out_file.close()

if __name__ == "__main__":
    main(None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model_AO', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1935c9e8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model_AO/model.ckpt-97701
INFO:tensorflow:Saving checkpoints for 97702 into /tmp/mnist_convnet_model_AO/model.ckpt.
IN